In [ ]:
from IPython.core.display import HTML
with open('./style.css') as f:
    css = f.read()
HTML(css)

# Play Chess Endgames Using Gaviota Endgame Tablebase

## What Is This Jupyter Notebook About?
The following notebook can be used to play any chess endgame with up to 4 pieces. To simulate the chess game itself the Python library [chess](https://python-chess.readthedocs.io/en/latest/) is used. Each game is exported and may be re-imported to retrace the game. The moves of white can be determined by yourself or randomly and the moves of black are determined with the help of the [Gaviota endgame tablebases](https://www.chessprogramming.org/Gaviota_Tablebases) developed by Miguel A. Ballicora. The tablebases are probed using a C-based API to determine the depth-to-mate (i.e., the number of half-moves until a player is mated) for a given situation on a chessboard and using this depth-to-mate information, the best possible move for black is played. In general, Gaviota supports endgames with up to 5 pieces. However, this implementation only includes the tablebases with up to 4 pieces.

The basic idea is to declare the ChessEndgame class first and then create it incrementally, so that the notebook is easy to understand and nevertheless a complete class is created. This is necessary because we want to override / extend the methods of this created class in a subsequent notebook, so not only the Gaviota tablebases, but custom tablebases, which were created by means of the so-called retrograde analysis, can be used.

## Imports And Preperations

In [ ]:
import chess                                       # Simulate the chess game
import chess.gaviota                               # Load Gaviota Tablebase
from IPython.display import display, clear_output  # Better visualization and display of the chess board
import random                                      # Random moves and random creation of endgame positions
from datetime import datetime                      # Assign exported games with exact date
from pytz import timezone                          # Assign exported games with exact date
from typing import Union, List, Set, Dict          # Types to enable direct method signatures

In [ ]:
class Mate(Exception): pass                        # Exception to intercept checkmate or stalemate
class InvalidConfig(Exception): pass               # Exception to intercept an incorrect configuration

In [ ]:
class ChessEndgame(): pass                         # Class, which will be used to play and import chess endgames.
class CustomTablebase(): pass                      # Custom Tablebase Class, which will be specified in a later notebook. Used here for typing only.

## Playing A Game

### Main Method

**`play_game(endgame_config: Union[str, List[chess.Piece], bool], tablebase_config: Dict[str, Union[str, bool]], export_only: bool) -> None`**

This is the main method which is called to play a chess end game. The following three paremeters must be specified:
1. `endgame_config` is the configuration of the end game. The options are as follows:

 - FEN or EPD as `str`: e.g. `'1K6/8/8/8/8/1r2k3/8/8 w - - 0 1'`  
 - Chess pieces as `str` (pieces are placed on random positions): e.g. `'Kkr'`  
 - Chess pieces as `List` (pieces are placed on random positions): e.g. `[chess.Piece(chess.KING, chess.WHITE), chess.Piece(chess.KING, chess.BLACK), chess.Piece(chess.ROOK, chess.BLACK)]`  
 - Winning color as `bool` (random predefined pieces are placed on random positions): e.g. `chess.BLACK` resp. `False`


2. `tablebase_config` is the configuration of the tablebase to be used: e.g. `{'directory': './gaviota', 'custom': False}`


3. `export_only` is flag, which decides whether the game should be displayed or just exported: e.g. `False`
 
 
If the game is played and displayed normally, the player should be asked before each white move which move should be executed. Either a move in UCI format (e.g. a1a2) can be entered or the field can be left empty to execute a random move. As already mentioned, the black moves are executed automatically based on the tablebase. The game then runs until the board is checkmate or stalemate.

The export file contains the endgame in a custom format. At the beginning there is an EPD string which describes the endgame situation. Separated by commas, the half-moves of white and black follow in UCI format until the board is checkmate or stalemate, e.g. `8/8/3k4/1r6/8/8/1K6 w - -, b1a1, d6e6, a1a2, e6d5, a2a1, d5d4, a1a2, d4c3, a2a1, c3c2, a1a2, b5a5, Checkmate`. Alternatively, the `.pgn format` (Portable Game Notation) could have been used. However, this assumes a standard board as the starting node, so we used our own format for our specific usecase, which is easily readable. The import of such a file is done by the function `import_game` at the end of this notebook.


In [ ]:
def play_game(self, endgame_config: Union[str, List[chess.Piece], bool], tablebase_config: Dict[str, Union[str, bool]], export_only: bool) -> None:
    export_file = f'games/{datetime.now(timezone("Europe/Berlin")).strftime("%d-%m-%Y_%H_%M_%S_%f")}.txt'
    with open(export_file, 'w') as export:
        board     = self._load_endgame_config(endgame_config)
        tablebase = self._open_tablebase(tablebase_config, board)
        export.write(f'{board.epd()}, ')
        try:
            self._check_for_mates(board)
            while True:
                if not export_only: display(board)
                self._white_move(board, export_only)      #Asks user for legal move, random move or autoplay and executes it on the board
                export.write(f'{board.peek()}, ')
                self._check_for_mates(board)
                if not export_only: clear_output()
                if not export_only: display(board)
                self._black_move(board, tablebase)        #Gets DTMs from tablebase and executes the best legal move for black on the board
                export.write(f'{board.peek()}, ')
                self._check_for_mates(board)
        except Mate as mate:
            export.write(str(mate))
            if not export_only: 
                display(board)
                print(str(mate))
            print(f"Saved game at {export_file}")
        finally:
            tablebase.close()
            
setattr(ChessEndgame, 'play_game', play_game)
del play_game

### Auxiliary Methods

In order to increase the readabilty of the main method, it uses several auxiliary methods which will be explained below in further detail. These are also added to the ChessEndgame class as private methods:
- **`load_endgame_config(endgame_config: Union[str, List[chess.Piece]]) -> chess.Board`**
- **`open_tablebase(tablebase_config: Dict[str, Union[str, bool]]) -> Union[chess.gaviota.NativeTablebase, CustomTablebase]`**
- **`check_for_mates(board: chess.Board) -> None`**
- **`white_move(board: chess.Board, export_only: bool) -> None`**
- **`black_move(board: chess.Board, tablebase: Union[chess.gaviota.NativeTablebase, CustomTablebase]) -> None`**

**Remark:** Direct data encapsulation does not exist in Python. So all of the methods can be called on an instance of ChessEndgame via their name. But by convention, however, methods and attributes beginning with an underscore are considered private and should not be used directly. Alternatively, the methods could have been made to start with a dunder (double underscore). This way, Python name mangling intervenes and the methods are textually replaced with `_classname__method`. Thus, for example, load_engame_config could no longer be called by `instance._load_engame_config`, but only by `instance._ChessEndgame__load_endgame_config`. However, this name mangling is mainly to prevent name collisions with subclasses and would only complicate the incremental creation of class methods. For this reason, data encapsulation by convention is sufficient in our case.

#### Loading The Endgame Config

**`load_endgame_config(endgame_config: Union[str, List[chess.Piece]]) -> chess.Board`**

The first auxiliary method is called `load_endgame`. This is used to convert the `endgame_config` given to the main method into a corresponding chess board. For this the 4 cases described above are distinguished. FEN or EPD strings are simply passed to the `chess.board` constructor. The specification of the chess pieces as string or list uses another auxiliary function: `piece_str_to_board`. This generates random squares for the given pieces and sets them to the squares. By using a set as data structure for the squares it is ensured that each piece is assigned a different square and thus no square is overwritten. Finally, the pieces are combined with the respective fields and placed on the board via a dictionary / piece map.The last case, where only one color is specified, which should have an advantage, is also used the previously described `piece_str_to_board` function.  Before that, however, a random position is selected from predefined interesting positions. If the declared cases are not met, an exception is raised which interrupts the program call and informs the end user that the `endgame_config` was not specified correctly and thus cannot be interpreted correctly.

In [ ]:
def load_endgame_config(self, endgame_config: Union[str, List[chess.Piece]]) -> chess.Board:
    if isinstance(endgame_config, str) and len(endgame_config) >= 5: #FEN or EPD as str
        return chess.Board(endgame_config)
    elif isinstance(endgame_config, str) and len(endgame_config) < 5: #Chess pieces as str
        return self._piece_str_to_board(endgame_config)
    elif isinstance(endgame_config, list): #Chess pieces as list
        return self._piece_str_to_board(''.join(piece.symbol() for piece in endgame_config))
    elif isinstance(endgame_config, bool): #Winning color as bool
        if endgame_config == chess.WHITE:  piece_strs = ['KRk', 'KQk', 'KBBk', 'KNNk', 'KBNk', 'KQkr']
        else:                              piece_strs = ['Kkr', 'Kkq', 'Kkbb', 'Kknn', 'Kkbn', 'KRkq']
        return self._piece_str_to_board(random.choice(piece_strs))
    raise InvalidConfig(f"Type {type(endgame_config)} as datatype for endgame_config is not supported.")
    
setattr(ChessEndgame, '_load_endgame_config', load_endgame_config)
del load_endgame_config

In [ ]:
def piece_str_to_board(self, piece_str: str) -> chess.Board:
    board = chess.Board()
    board.clear()
    while not board.is_valid() or board.is_checkmate():
        squares = set()
        while not len(squares) == len(piece_str):
            squares.add(random.randint(chess.A1, chess.H8))
        board.set_piece_map(dict(zip(squares, (chess.Piece.from_symbol(symbol) for symbol in piece_str))))
    return board

setattr(ChessEndgame, '_piece_str_to_board', piece_str_to_board)
del piece_str_to_board

#### Opening The Tablebase

**`open_tablebase(self, tablebase_config: Dict[str, Union[str, bool]], board: chess.Board) -> Union[chess.gaviota.NativeTablebase, CustomTablebase]`**

This method takes care of opening the specified tablebase. If the parameter `custom` is set to True, it tries to open a custom tablebase under the `directory` path. This will be implemented in a subsequent notebook and at the moment an Exception is raised which interrupt the programm call and informs the end user that custom tablebases are not implemented yet. If the parameter is set to False, a Gaviota tablebase is opened using the given path. The opened tablebase is finally returned. 

In [ ]:
def open_tablebase(self, tablebase_config: Dict[str, Union[str, bool]], board: chess.Board) -> Union[chess.gaviota.NativeTablebase, CustomTablebase]:
    if tablebase_config['custom']:
        raise InvalidConfig("Custom tablebases are not implemented yet.")
    else:
        return chess.gaviota.open_tablebase(tablebase_config['directory'])
    
setattr(ChessEndgame, '_open_tablebase', open_tablebase)
del open_tablebase

#### Raising Checkmate And Stalemate Exception

**`check_for_mates(board: chess.Board) -> None`**

The next method called `check_for_mates` is a simple query whether the board is checkmate / stalemate or not. It only gets the current board as a parameter. As you can see in the main method, this check is performed after every move, so that in case of checkmate / stalemate an exception directly ends the game and breaks out of the loop of the main method.

In [ ]:
def check_for_mates(self, board: chess.Board) -> None:
    if board.is_checkmate(): raise Mate('Checkmate')
    if board.is_stalemate(): raise Mate('Stalemate')
        
setattr(ChessEndgame, '_check_for_mates', check_for_mates)
del check_for_mates

#### Implementing Half Moves For White

**`white_move(board: chess.Board, export_only: bool) -> None`**

Next comes the auxiliary method `white_move`, which is responsible for the white half-moves. The current board and the `export_only` flag are given as parameters. If the latter is set to true, a random legal move is simply executed on the current board. However, if the flag is set to false, the user is asked which move to play for white. Entering an empty string (i.e. just Enter) will also execute a random move. Alternatively, a legal move can be entered in UCI format (e.g. a1a2). If the input has a wrong format or the move is not legal, the question is asked again until a legal move is entered.

In [ ]:
def white_move(self, board: chess.Board, export_only: bool) -> None:

    if export_only: 
        board.push(random.choice(list(board.generate_legal_moves()))) 
        return
    while True:
        print("Please insert your move in UCI format (e.g. a1a2) or leave blank to play a random move:")
        move_uci = input()
        if move_uci == '': 
            board.push(random.choice(list(board.generate_legal_moves()))) 
            return
        try:
            move = chess.Move.from_uci(move_uci)
            if move in board.generate_legal_moves():
                board.push(move)
                return
            print(f"{move_uci} is not a valid move.")
        except ValueError:
            print(f"{move_uci} is not in valid UCI format.")
            
setattr(ChessEndgame, '_white_move', white_move)
del white_move

#### Implementing Half Moves For Black

**`black_move(board: chess.Board, tablebase: Union[chess.gaviota.NativeTablebase, CustomTablebase]) -> None`**

Finally the auxiliary method `black_move` follows, which is responsible for the black half-moves. It also receives the current board as a parameter and the open tablebase, which is used to select the best move. For this, first a random move of the legal moves is executed and it is checked if this puts white in checkmate. If not, we request the depth to mate from the tablebase and run into a loop, which tries all legal moves on the same way. After each move we check if the depth-to-mate of the executed move is better than the depth to mate of the previous best move. Additionally, the check for checkmate is executed each time, since checkmate returns the same depth-to-mate as stalemate or balanced positions (0). So a move that checkmates white would not be distinguishable from very bad moves that lead to stalemate or an even position. However, all other moves can be compared by their depth-to-mate. This is done in the additional auxiliary method `is_better_move`. This gets two depth-to-mate as integer and tells whether the first or the second is better for black. The value depth-to-mate meets the following specification:

>Probes for depth to mate information.
>The absolute value is the number of half-moves until forced mate (or 0 in drawn positions). The value is positive if the side to move is winning, otherwise it is negative. (see [Docs](https://python-chess.readthedocs.io/en/latest/gaviota.html#chess.gaviota.PythonTablebase.probe_dtm))

In our concrete example, black half-moves are executed and then the depth-to-mate is queried. This means that it is white's move, so a positive depth-to-mate means that white can force a win in depth-to-mate half-moves. On the other hand, a negative value means that black can force a win in so many half-moves. 0 means either a position where no color can force a win, stalemate or checkmate (this is the reason why checkmate is queried beforehand). Better for black in this case means therefore a negative number as close to zero as possible but not equal to zero (checkmates are already detected beforehand as described above). This means we have three cases:
1. If the old value is positive, white is winning. Better moves would be:
 - Any greater value than the current, which would increase the number of moves until forced mate
 - Any value of zero or lower: this would flip the game, by indicating either a draw, or that black is winning
2. If the old value is negative, black is winning. Better moves would be:
 - Any value closer to zero (greater than old value), but not zero (because we already checked for checkmate) and not positive
3. If the old value is zero, the game is facing a draw. Better moves would be:
 -  Any value less than zero, which would mean that black is winning

In [ ]:
def black_move(self, board: chess.Board, tablebase: Union[chess.gaviota.NativeTablebase, CustomTablebase]) -> None:
    legal_moves = board.generate_legal_moves()
    move = next(legal_moves)
    board.push(move)
    if board.is_checkmate(): 
        return
    best_move = (move, tablebase.probe_dtm(board))
    board.pop()
    for move in legal_moves:
        board.push(move)
        if board.is_checkmate(): 
            return
        dtm = tablebase.probe_dtm(board)
        if self._is_better_move(dtm, best_move[1]):
            best_move = (move, dtm)
        board.pop()

    board.push(best_move[0])
    return

setattr(ChessEndgame, '_black_move', black_move)
del black_move

In [ ]:
def is_better_move(self, new: int, old: int) -> bool:
    if old > 0:
        if new > 0:
            return new > old
        elif new < 0:
            return True
        else:
            return True
    elif old < 0:
        if new > 0:
            return False
        elif new < 0:
            return new > old
        else:
            return False
    else:
        if new > 0:
            return False
        elif new < 0:
            return True
        else:
            return False
        
setattr(ChessEndgame, '_is_better_move', is_better_move)
del is_better_move

## Importing A Game From File

Finally, we define the method `import_game`, which is only used to import a game that has already been played. For this we read in the specified file, load the board based on the EPD string, which is located at the beginning. After that, each move, which was stored in uci format and separated by commas, is executed until Checkmate or Stalemate is reached. After each step the board is printed. If there is no checkmate or stalemate at the end of the file, an exception is raised that file contains an unfinished game.

In [ ]:
def import_game(self, import_file: str) -> None:
    with open(import_file, 'r') as game:
        steps = game.readline().rstrip().split(', ')
        if steps[-1] not in ['Checkmate', 'Stalemate']:
            print('Unfinished game file')
            return
        board = chess.Board(steps[0])
        display(board)
        for step in steps[1:]:
            if step in ['Checkmate', 'Stalemate']:
                print(step)
                return
            board.push(chess.Move.from_uci(step))
            display(board)
            
setattr(ChessEndgame, 'import_game', import_game)
del import_game

To test the whole ChessEndgame class the separate notebook `PlayChessEndgame.ipynb` should be used. This is the case because the class ChessEndgame is extended in a subsequent notebook and thus included by means of the magic command `%run ChessEndgame.ipynb`. If there are cells for playing in this notebook, they would be unintentionally executed in the notebook that extends the class. A workaround would be the integration by means of nbimporter whereby only explicit classes can be imported without the entire notebook being executed. This is not possible in this case, however, because the class is created incrementally for a better overview and thus importing the explicit class only imports the empty class of the notebook start.